# PyTerrier Starter: Re-Rank with PyTerrier for Task 2 in Touche 2023

This is the [PyTerrier](https://github.com/terrier-org/pyterrier) baseline for [task 2 on Evidence Retrieval for Causal Questions](https://touche.webis.de/clef23/touche23-web/evidence-retrieval-for-causal-questions.html) in [Touché 2023](https://touche.webis.de/clef23/touche23-web/).

This notebook implements a simple BM25 re-ranker.

### Adapt the notebook locally

You can adapt/run this baseline locally with docker and can directly deploy and run it in [TIRA.io](https://www.tira.io/task/touche-2023-task-2).

With docker installed, you can start this notebook with the command:

```
docker run --rm -ti \
    -p 8888:8888 \
    -v ${PWD}:/workspace \
    webis/tira-touche23-task-2-pyterrier-baseline:0.0.2 \
    jupyter-lab --allow-root --ip 0.0.0.0
```

### Deployment in TIRA

To deploy approaches in TIRA, you upload the image and specify the command that is to be executed in the image. TIRA gives you a personalized documentation on how to upload the image, and to run this notebook in TIRA you can specify the following command in TIRA:

```
/workspace/run-pyterrier-notebook.py --notebook /workspace/re-rank-pipeline.ipynb --input $inputDataset --output $outputDir
```

You can dry-run this on your machine by executing the command:

```
tira-run \
    --input-directory ${PWD}/sample-input/re-rank \
    --image webis/tira-touche23-task-2-pyterrier-baseline:0.0.2 \
    --command '/workspace/run-pyterrier-notebook.py --notebook /workspace/re-rank-pipeline.ipynb --input $inputDataset --output $outputDir'
```

In this example above, the command `/workspace/run-pyterrier-notebook.py --notebook /workspace/re-rank-pipeline.ipynb --input $inputDataset --output $outputDir` is the command that you would enter in TIRA, and the `--input-directory` flag points to the inputs.

This creates a run file `tira-output/run.txt`, with content like (`cat sample-output/run.txt |head -3`):

```
1111 NEU clueweb12-1106wb-16-17437 1 -1.6923219067023894 bm25-re-rank-baseline
1111 NEU clueweb12-1212wb-00-02238 2 -2.768819661029173 bm25-re-rank-baseline
1111 NEU clueweb12-0302wb-19-28258 3 -2.768819661029173 bm25-re-rank-baseline
```



### Additional Resources

- The [PyTerrier tutorial](https://github.com/terrier-org/ecir2021tutorial)
- The [PyTerrier documentation](https://pyterrier.readthedocs.io/en/latest/)
- The [TIRA quickstart](https://touche.webis.de/clef23/touche23-web/evidence-retrieval-for-causal-questions.html#tira-quickstart)



### Step 1: Import everything and load variables

In [1]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run, load_rerank_data
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('./sample-input/re-rank')

df_rerank = load_rerank_data('./sample-input/re-rank')
df_rerank['query'] = df_rerank['query'].apply(lambda i: "".join([x if x.isalnum() else " " for x in i]))

Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


I will use a small hardcoded example located in ./sample-input/re-rank.
The output directory is /tmp/
I will use a small hardcoded example located in ./sample-input/re-rank.
The output directory is /tmp/


In [2]:
df_rerank.head(3)

,qid,query,docno,text
0,1111,does computer work increase eye pressure,clueweb12-1106wb-16-17437,Eyes hurt looking computer screen\n\n\n\nEyes ...
1,1111,does computer work increase eye pressure,clueweb12-0302wb-19-28258,How the Eye Works\n\n\n\nHow the Eye Works\n\n...
2,1111,does computer work increase eye pressure,clueweb12-1212wb-00-02238,How the Eye Works\n\n\n\nHow the Eye Works\n\n...


### Step 2: Define the actual retrieval appraoch

In [3]:
bm25_scorer = pt.text.scorer(body_attr="text", wmodel='BM25', verbose=True)


### Step 3: Run the pipeline

In [4]:
run = bm25_scorer(df_rerank)
run.head(3)

31documents [00:00, 51.98documents/s]                                                                                                             
BR(BM25):   0%|                                                                                                              | 0/3 [00:00<?, ?q/s]

08:40:23.375 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


BR(BM25): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 23.14q/s]


,qid,docno,text,rank,score,query
0,1111,clueweb12-1106wb-16-17437,Eyes hurt looking computer screen\n\n\n\nEyes ...,0,-1.692322,does computer work increase eye pressure
1,1111,clueweb12-0302wb-19-28258,How the Eye Works\n\n\n\nHow the Eye Works\n\n...,1,-2.768820,does computer work increase eye pressure
2,1111,clueweb12-1212wb-00-02238,How the Eye Works\n\n\n\nHow the Eye Works\n\n...,2,-2.768820,does computer work increase eye pressure


### Step 4: Stence Detection

In [6]:
print('Step 4: Define stence detection')

def detect_stance(query_document_pair):
    # As baseline, we return always neutral
    return 'NEU'

run['Q0'] = run.apply(lambda i: detect_stance(i), axis=1)


Step 4: Define stence detection


### Step 5: Persist results

In [8]:
print('Step 5: Persist Run.')

persist_and_normalize_run(run, 'bm25-re-rank-baseline', output_file=output_directory + '/run.txt')

print('Done...')

Step 5: Persist Run.
Done...


In [9]:
!head -3 {output_directory}/run.txt

1111 NEU clueweb12-1106wb-16-17437 1 -1.6923219067023894 bm25-re-rank-baseline
1111 NEU clueweb12-1212wb-00-02238 2 -2.768819661029173 bm25-re-rank-baseline
1111 NEU clueweb12-0302wb-19-28258 3 -2.768819661029173 bm25-re-rank-baseline
